In [ ]:
from IPython.display import HTML




In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import ipywidgets as widgets
from IPython.display import display, HTML
import pandas as pd
import os

# File path for the CSV
csv_file_path = "cix.csv"

# Create the necessary widgets
string_box = widgets.Text(description='Ticker')
description_box = widgets.Text(description='Description')
save_button = widgets.Button(description='Save')
my_cix_button = widgets.Button(description="My Cix's")

output_save = widgets.Output()
output_display = widgets.Output()

# Function to save the entries to the CSV file
def save_to_csv(b):
    with output_save:
        output_save.clear_output()
        string = string_box.value
        description = description_box.value

        # Create a DataFrame from the inputs
        df = pd.DataFrame([[string, description]], columns=['String', 'Description'])

        # Save to CSV
        if os.path.exists(csv_file_path):
            df.to_csv(csv_file_path, mode='a', header=False, index=False)
        else:
            df.to_csv(csv_file_path, index=False)

        #print(f'Saved: {string}, {description}')

# Function to display the contents of the CSV file
def display_cix(b):
    with output_display:
        output_display.clear_output()
        if os.path.exists(csv_file_path):
            df = pd.read_csv(csv_file_path)
            # Center the DataFrame display
            styled_table = df.style.set_table_styles([{
                'selector': 'table',
                'props': [('margin-left', 'auto'), ('margin-right', 'auto'), ('text-align', 'center')]
            }]).to_html()

            centered_html = f'<div class="centered-output"><div class="centered-table">{styled_table}</div></div>'
            display(HTML(centered_html))
        else:
            print('No entries found.')

# Set button click events
save_button.on_click(save_to_csv)
my_cix_button.on_click(display_cix)

# Display widgets
display(string_box, description_box, save_button, output_save, my_cix_button, output_display)

# Load custom CSS for centering the table
display(HTML('''
<style>
.centered-output {
    display: flex;
    justify-content: center;
    align-items: center;
    width: 100%;
    margin-top: 20px;
}
.centered-table {
    margin: 0 auto;
    text-align: center;
}
</style>
'''))


Text(value='', description='Ticker')

Text(value='', description='Description')

Button(description='Save', style=ButtonStyle())

Output()

Button(description="My Cix's", style=ButtonStyle())

Output()

In [3]:
import pandas as pd
from xbbg import blp
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime as dt
from datetime import timedelta as td
import statsmodels.api as sm
import re
import matplotlib.image as image
from ipywidgets import interact, IntSlider, Checkbox, Dropdown, Output, HBox, VBox, interactive, interactive_output, ToggleButton,Text, Button, DatePicker, IntText, ToggleButtons, RadioButtons,SelectMultiple
from IPython.display import display, clear_output
import itertools
from scipy import stats
from scipy.optimize import minimize 
from scipy.special import ndtr

spot_library = {'eur_6':['EUSA', ' Curncy'],'eur': ['EESWE', ' Curncy'], 'usd': ['USOSFR', ' Curncy'], 
                'gbp': ['BPSWS', ' Curncy'],'chf': ['SFSNT', ' Curncy'],'sek': ['SKSW', ' Curncy'],
                 'nok': ['NKSW', ' Curncy'], 'hkd': ['HDSW', ' Curncy'],'czk': ['CKSW', ' Curncy'],
                 'pln': ['PZSW', ' Curncy'],'ils':['ISSW', ' Curncy'],  'cad':['CDSW', ' Curncy'], 
                 'jpy':['JYSO', ' Curncy'], 'aud': ['ADSW', ' Curncy'],'sgd':['SDSW', ' Curncy'],
                'krw': ['KWSWNI', ' Curncy'],
                'zar': ['SASW', ' Curncy'],
                'nzd': ['NDSW', ' Curncy'],
                'mxn': ['MPSW', ' Curncy']} 

forward_library = {'eur_6': ['EUSA', ' Curncy'], 
                 'eur': ['S0514FS ', ' BLC Curncy'], 
                 'usd': ['S0490FS ', ' BLC Curncy'], 
                 'gbp': ['S0141FS ', ' BLC Curncy'],
                 'chf': ['S0234FS ', ' BLC Curncy'],
                 'sek': ['SD0020FS ', ' BLC Curncy'],
                 'nok': ['SD0313FS ', ' BLC Curncy'],
                 'hkd': ['HDFS', ' Curncy'],
                 'czk': ['S0320FS ', ' BLC Curncy'],
                 'pln': ['S0323FS ', ' BLC Curncy'],
                 'ils': ['ISFS', ' Curncy'],
                 'cad': ['CDFS', ' Curncy'],
                 'jpy': ['S0195FS ', ' BLC Curncy'],
                 'aud': ['SD0302FS ', ' BLC Curncy'],
                'sgd': ['SDFS', ' Curncy'],
                'krw': ['S0205FS ', ' BLC Curncy'],
                'zar': ['SAFS', ' Curncy'],#
                'nzd': ['SD0015FS ', ' BLC Curncy'],
                'mxn': ['SD0083FS ', ' BLC Curncy']} 

################################################# SWAP STRUCTURE BUILDERS ################################################

def cut(some):
    x = 0
    y = some[0]
    while y.isdigit():
        y=some[x]
        x+=1
    return some[:x],some[x:]


def f(tenor):
    x = re.findall(r'\d+',tenor)[0]
    num = ''
    if 'm' in tenor.lower():
        if int(x) // 12 >0:
            num+='1'
        num+=chr(64+(int(x)%12))
    else:
        if len(x) == 1:
            num+='0'
        num +=x
    return num

def t(tenor):
    x = re.findall(r'\d+',tenor)[0]
    num = ''
    if 'm' in tenor.lower():
        if int(x) // 12 >0:
            num+='1'
        else:
            num+='0'
        num+=chr(64+(int(x)%12))
    else:
        if len(x) == 1:
            num+='0'
        num +=x
    return num


def spot_ticker(dex,tenor):
    if dex.lower() == 'mxn':
        y = int(tenor[:-1]) * 13
        num = f'{y//12}{chr(64+(y%12))}'
    else:
        num = tenor[:-1]
    return f'{spot_library[dex][0]}{num}{spot_library[dex][1]}'

def forward_ticker(dex,fwd):
    dex = dex.lower()
    fwd = fwd.lower()
    old = ['eur_6','hkd','ils','cad','sgd','zar']
    
    if cut(fwd)[0] == '0y':
        return spot_ticker(dex,cut(fwd)[1])
    elif dex == 'eur_6':
        F,T = f(cut(fwd)[0]),t(cut(fwd)[1])
        return f'{forward_library[dex][0]}{F}{T}{forward_library[dex][1]}'
    elif dex in old:
        F,T = t(cut(fwd)[0]),t(cut(fwd)[1])
        return f'{forward_library[dex][0]}{F}{T}{forward_library[dex][1]}'
    else:
        
        return f'{forward_library[dex][0]}{fwd.upper()}{forward_library[dex][1]}'


def swap_structure(dex,structure,start, end = 'today',bps = True):
    f_fly = structure.count('/') == 2
    f_crv = structure.count('/') == 1  
    fly = structure.count('s') == 3
    crv = structure.count('s') == 2
    out = max(2 -sum([i.isalpha() for i in structure]),0)
    
    if f_fly or f_crv:
        legs = [forward_ticker(dex,i) for i in structure.split('/')]
    elif fly or crv:
        legs = [spot_ticker(dex,i) for i in [i+'Y' for i in structure.split('s') if i.isdigit()]]
    else:
        legs = forward_ticker(dex,('0Y'*out) + structure)
        
    df = blp.bdh(legs, flds='px_last', start_date=start,end_date=end).fillna(method ='bfill')  *(100 if bps else 1)
    s  = pd.DataFrame({})
    if f_fly or fly:
        x = (2 * df.iloc[:,1]) - (df.iloc[:,0] + df.iloc[:,2])
    elif f_crv or crv:
        x = df.iloc[:,1] - df.iloc[:,0]
    else:
        x = df.iloc[:,0]
        
    s[f'{dex.upper()} {structure}'] = x
    return s

################################################# BOND STRUCTURE BUILDERS ################################################
def bond_ticker(bond_name, bond_tenor):
    cmb = {'ust':1,'dbr':2,'spgb':8,'frtr':4,'btp':5,'ukt':6}
    return f'RV000{cmb[bond_name]}P {bond_tenor.upper()} BLC Curncy'

def bond_structure(bonds,structure, start, end = 'today'):
    if '-' in bonds:
        s = pd.DataFrame({})
        bond1,bond2 = bonds.split('-')
        s[f'{bonds.upper()} {structure}'] = bond_structure(bond1,structure, start, end)[f'{bond1.upper()} {structure}'] - bond_structure(bond2,structure, start, end)[f'{bond2.upper()} {structure}']
        return s
    else:
        if 'y' in structure.lower(): #bond o/r
            tickers = bond_ticker(bonds.lower(), structure)
            base = blp.bdh(tickers, 'px_last', start, end).fillna(method ='bfill') * 100
            base.columns = [f'{bonds.upper()} {structure}']
            return base
        
        elif structure.lower().count('s') == 2: #bond curve
            tickers = [bond_ticker(bonds.lower(), f'{i}y') for i in structure.lower().split('s') if i != '']
            base = blp.bdh(tickers, 'px_last', start, end).fillna(method ='bfill') * 100
            base.columns = tickers
            s = pd.DataFrame({})
            s[f'{bonds.upper()} {structure}'] = base[tickers[1]] - base[tickers[0]]
            return s
        
        elif structure.lower().count('s') == 3: # bond fly
            tickers = [bond_ticker(bonds.lower(), f'{i}y') for i in structure.lower().split('s') if i != '']
            base = blp.bdh(tickers, 'px_last', start, end).fillna(method ='bfill') * 100
            base.columns = tickers
            s = pd.DataFrame({})
            s[f'{bonds.upper()} {structure}'] = (2*base[tickers[1]]) - (base[tickers[0]] + base[tickers[2]])
            return s

################################################# FUTURES STRUCTURE BUILDERS ################################################

def fut_ticker(dex, contract, name = False):
    x  = ''
    if dex.lower()[:3] == 'eur':
        x = 'ER'
    elif dex.lower() == 'usd':
        x = 'SFR'
    elif dex.lower() == 'gbp':
        x = 'SFI'
    
    if name:
        return x
    else:
        return f'{x}{contract} Comdty'   
           
    
def fut_structure(dex,structure, start, end = 'today'):
    name = fut_ticker(dex.lower(), 0,True)
    
    if structure.lower().count('s') == 2: #fut curve
        tickers = [fut_ticker(dex.lower(), int(i)) for i in structure.lower().split('s') if i != '']
        base = blp.bdh(tickers, 'px_last', start, end).fillna(method ='bfill') * 100
        base.columns = tickers
        s = pd.DataFrame({})
        s[f'{name}{structure.replace("s","")}'] = base[tickers[0]] - base[tickers[1]]
        return s
        
    elif structure.lower().count('s') == 3: # fut fly
        tickers = [fut_ticker(dex.lower(), int(i)) for i in structure.lower().split('s') if i != '']
        base = blp.bdh(tickers, 'px_last', start, end).fillna(method ='bfill') * 100
        base.columns = tickers
        s = pd.DataFrame({})
        s[f'{name}{structure.replace("s","")}'] = (2*base[tickers[1]]) - (base[tickers[0]] + base[tickers[2]])
        return s
    else:
        tickers = fut_ticker(dex.lower(), structure)
        base = blp.bdh(tickers, 'px_last', start, end).fillna(method ='bfill') * 100
        base.columns = [f'{name}{structure}']
        return base

################################################# ASW STRUCTURE BUILDERS ################################################

def get_asw(bond_name,tenor,start, end ='today',euro = 'estr'):
    asso_swap = {'ust':'usd','dbr':'eur','spgb':'eur','frtr':'eur','btp':'eur','ukt':'gbp'}
    
    if asso_swap[bond_name] == 'eur' and euro != 'estr':
        tickers = [bond_ticker(bond_name,tenor), spot_ticker(asso_swap[bond_name]+'_6',tenor)]
    else:
        tickers = [bond_ticker(bond_name,tenor), spot_ticker(asso_swap[bond_name],tenor)]
    base = blp.bdh(tickers, 'px_last', start, end).fillna(method ='bfill') * 100
    base.columns = tickers
    s = pd.DataFrame({})
    s[f'{tenor.upper()} {bond_name.upper()} ASW'] = base[tickers[1]] - base[tickers[0]]
    return s

def asw_structure(bonds,structure, start, end = 'today',euro = 'estr'):
    if '-' in bonds:
        s = pd.DataFrame({})
        bond1,bond2 = bonds.split('-')
        s[f'{bonds.upper()} ASW {structure}'] = asw_structure(bond1,structure, start, end,euro)[f'{bond1.upper()} ASW {structure}'] - asw_structure(bond2,structure, start, end,euro)[f'{bond2.upper()} ASW {structure}']
        return s
    else:
        if 'y' in structure.lower(): #asw o/r
            return get_asw(bonds,structure,start, end,euro)
        
        elif structure.lower().count('s') == 2: #asw curve
            legs = [ f'{i}y' for i in structure.lower().split('s') if i != '']
            short_leg, long_leg = [get_asw(bonds,i,start,end,euro) for i in legs]
            df = short_leg.join(long_leg, how = 'inner')
            s = pd.DataFrame({})
            s[f'{bonds.upper()} ASW {structure}'] = df.iloc[:,1]  - df.iloc[:,0]  
            return s
        
        elif structure.lower().count('s') == 3: # asw fly
            legs = [ f'{i}y' for i in structure.lower().split('s') if i != '']
            short_leg, belly ,long_leg = [get_asw(bonds,i,start,end,euro) for i in legs]
            short_leg = short_leg.join(belly, how = 'inner')
            df = short_leg.join(long_leg, how = 'inner')
            s = pd.DataFrame({})
            s[f'{bonds.upper()} ASW {structure}'] = (2 * df.iloc[:,1])  - (df.iloc[:,0] + df.iloc[:,2])  
            return s
        

######################################### TICKER HANDLERS #################################################################

def get_other(ticker,attribute,title,start,end='today' ):
    if 'price' in attribute.lower():
        attribute = 'px_last'
    else:
        attribute = 'YLD_YTM_MID'
    base = blp.bdh(ticker,attribute,start,end)
    base.columns = [title]
    return base

######################################### INVOICE SPREAD STRUCTURE BUILDERS ################################################


def get_ivsp(country,start, end ='today',which = 'TUA',euro ='estr'):
    tkr = {'USD':['TUAISPS','3YAISPS','FVAISPS','TYAISPS','UXYAISPS','USAISPS','WNAISPS'],
           'GER':['DUAISP','OEAISP','RXAISP','UBAISP'],
           'ITA':['BTAISP','IKAISP'],
           'FRA':['OATAISP'],
           'GBP':['GAISPO']}
    
    if country.upper() == 'USD' or country.upper() == 'GER' or country.upper() == 'ITA' :
        for i in tkr[country.upper()]:
            if which in i:
                future = i
    else:
        future = tkr[country.upper()][0]
    
    if country.lower() in ['ger','ita','fra'] and euro == 'estr':
        future += 'E'
    final_ticker = f'{future} Comdty'
    base = blp.bdh(final_ticker,'px_last',start,end)
    base.columns = [f'{future} Comdty']    
    return base
                
######################################## MEETING DATES HANDLER ##################################################################
    
def md_swap_ticker(bank, meeting,m_of_m =False):
    central_bank = {'FOMC':['USSOFED',' Curncy'],
                'ECB':['EESF','A Curncy'],
                'MPC':['GPSF','A Curncy']}
    ticker = f'{central_bank[bank.upper()][0]}{meeting}{central_bank[bank.upper()][1]}' 
    if m_of_m:
        return blp.bdp(ticker, 'SW_EFF_DT').values.flatten()[0].strftime('%b')
    else:
        return ticker
    
def get_md_swap(bank,structure,start,end  = 'today'):
    if structure.lower().count('s') == 2:
        tickers = [md_swap_ticker(bank.upper(),int(i)) for i in structure.lower().split('s') if i != ''] 
        lbls = ''.join([md_swap_ticker(bank.upper(),int(i), True) for i in structure.lower().split('s') if i != ''])
        base = blp.bdh(tickers, 'px_last', start, end).fillna(method ='bfill') * 100
        base.columns = tickers
        s = pd.DataFrame({})
        s[f'{lbls} {bank.upper()}'] = base[tickers[1]] - base[tickers[0]]
        return s
        
    if structure.lower().count('s') == 3:
        tickers = [md_swap_ticker(bank.upper(),int(i)) for i in structure.lower().split('s') if i != '']
        lbls = ''.join([md_swap_ticker(bank.upper(),int(i), True) for i in structure.lower().split('s') if i != ''])            
        base = blp.bdh(tickers, 'px_last', start, end).fillna(method ='bfill') * 100
        base.columns = tickers
        s = pd.DataFrame({})
        s[f'{lbls} {bank.upper()}'] = (2*base[tickers[1]] )- (base[tickers[0]] + base[tickers[2]])
        return s
    else:
        tickers = md_swap_ticker(bank.upper(), structure)
        base = blp.bdh(tickers, 'px_last', start, end).fillna(method ='bfill') * 100
        base.columns = [f'{md_swap_ticker(bank.upper(), structure,True)}{bank.upper()}']
        return base

    
################################################# FX HANDLER ############################################################
def get_fx_pair(pair,start,end='today'):
    ticker = f'{pair.upper()} Curncy'
    base = blp.bdh(ticker,'px_last',start,end)
    base.columns = [pair.upper()]
    return base

######################################## BOND FUTURE HANDLER ##################################################################
def get_bondf_ticker(name):
    tic_dic = {'DU':'DUA Comdty','OE':'OEA Comdty','RX': 'RXA Comdty','UB': 'UBA Comdty',
                'BTS': 'BTSA Comdty','IK': 'IKA Comdty',
                'OAT': 'OATA Comdty',
                'G':'G A Comdty',
                'TU': 'TUA Comdty','FV': 'FVA Comdty', 'TY': 'TYA Comdty','WN': 'WNA Comdty','UXY': 'UXYA Comdty'}
    return tic_dic[name.upper()]
    

def get_bond_fut(_str, start,end = 'today', fld = 'yield'):
    fld = 'YLD_YTM_MID' if fld.lower() == 'yield' else 'PX_LAST' 
    _str = _str.upper()
    label = _str.split('/')
    legs = [get_bondf_ticker(i) for i in label]
    bps = 100 if fld == 'YLD_YTM_MID' else 1
    base  = blp.bdh(legs,fld,start,end) * bps
    base.columns = label
    s = pd.DataFrame({})
    
    if _str.count('/') == 2:
        s[_str] = 2 * base[label[1]] - (base[label[0]] + base[label[2]]) 
            
    elif _str.count('/') == 1:
        s[_str] =  base[label[1]] - base[label[0]] 
                                        
    else:
        s = base
        
    return s
    

######################################## INFL SWAP HANDLER ##################################################################



######################################## MASTER HANDLER ##################################################################
def get_cix_options():
    if os.path.exists('cix.csv'):
        df = pd.read_csv('cix.csv')
        return df.iloc[:,0].tolist()
    else:
        return []

def function_for_bb(trades,_from,_to):
    master_c = pd.DataFrame({})
    for i in trades.keys():
        if len(trades[i]) != 0:
            for j in trades[i]:
                if i == 'SWAP':
                    h = swap_structure(dex = j[0].lower() ,structure=j[1].lower() ,start = _from , end = _to)                                
                elif i == 'FUTURE':
                    h = fut_structure(dex = j[0].lower() ,structure=j[1].lower() ,start = _from , end = _to)                              
                elif i == 'BOND':
                    h = bond_structure(bonds = j[0].lower() ,structure=j[1].lower() ,start = _from , end = _to)
                elif i ==  'ASW':
                    h = asw_structure(bonds = j[0].lower() ,structure=j[1].lower() ,start = _from , end = _to, euro = j[2].lower())
                elif i == 'XMKT SWAP':
                    h1 = swap_structure(dex = j[0].lower() ,structure=j[1].lower() ,start = _from , end = _to)
                    h2 = swap_structure(dex = j[2].lower() ,structure=j[1].lower() ,start = _from , end = _to)
                    h_int = h1.join(h2, how = 'inner')
                    h = pd.DataFrame(h_int.iloc[:,0] - h_int.iloc[:,1], columns = [f'{j[0]}-{j[2]} {j[1]}'])
                elif i == 'MD SWAP':
                    h = get_md_swap(bank = j[0].upper(),structure=j[1].lower() ,start = _from , end = _to)
                elif i ==  'BOND FUTURE':
                    h = get_bond_fut(_str = j[0],start = _from , end = _to,fld =  j[1])
                elif i == 'FX':
                    h = get_fx_pair(pair = j[0].upper(),start = _from , end = _to)
                elif i == 'OTHER/CIX':
                    h = get_other(ticker = j[0], attribute= j[1], title = j[2], start = _from , end = _to)            
                elif i == 'MY CIX':
                    #print(j[0], j[0], j[0], _from, _to)
                    h = get_other(ticker = j[0], attribute= j[1], title = j[2], start = _from , end = _to)   
                if master_c.shape == (0,0):
                    master_c = h
                else:
                    master_c = master_c.join(h, how = 'inner')
    return master_c

In [4]:
from IPython.display import HTML, display
import ipywidgets as widgets

clrs = ['whites','reds','greens','blues']
def fut_presets(dex,clr):
    color = {key : range(1+(clrs.index(key)*4),5+(clrs.index(key)*4)) for key in clrs}
    fin_clr = []
    for i in clr.lower().split(', '):
        fin_clr += list(color[i])
    for i in fin_clr:
        yield (dex.upper(), str(i))

clr_dd = [', '.join([m.capitalize() for m in j]) for i in range(1,len(clrs)+1) for j in itertools.combinations(clrs, i) ]


sp_or_fw = ['Spot Tenors', 'Forward Gaps']
yc = ['(up to 10y)','(up to 15y)','(up to 30y)','(up to 50y)']
srt = ['Spot Curves', 'Spot Flys', 'Forward Curves']
swap_ops = [f'{i} {j}' for i in sp_or_fw for j in yc] + srt
bond_ops = ['Outright', 'Curves', 'Flys']

def swap_presets(dex, build):
    
    spot_front = [f'{i}y' for i in range (1,11)]
    spot_mid = spot_front + ['12y','15y']
    spot_back = spot_mid + ['20y','25y','30y']
    spot_ultraback =spot_back + ['40y','50y']
    
    fwds_front = [f'{i}y1y' for i in range (1,10)]
    fwds_mid = fwds_front +['10y2y','12y3y']
    fwds_back = fwds_mid + ['15y5y','20y5y','25y5y']
    fwds_ultraback = fwds_back + ['30y10y','40y10y']
    
    spot_curves = ['1s2s','2s5s','2s10s','5s10s','5s30s','10s30s']
    spot_flys = ['1s2s3s','1s3s5s', '2s5s10s', '5s10s30s']
    fwds_curves = ['1y1y/5y5y','1y2y/1y10y']
    answer = []
    if 'spot' in build.lower():
        if '10y' in build.lower():
            answer = spot_front
        elif '15y' in build.lower():
            answer = spot_mid
        elif '30y' in build.lower():
            answer = spot_back
        elif '50y' in build.lower():
            answer = spot_ultraback
        elif 'curves' in build.lower():
            answer = spot_curves
        elif 'flys' in build.lower():
            answer = spot_flys 
        else:
            answer = spot_curves + spot_flys
    else:
        if '10y' in build.lower():
            answer = fwds_front
        elif '15y' in build.lower():
            answer = fwds_mid
        elif '30y' in build.lower():
            answer = fwds_back
        elif '50y' in build.lower():
            answer = fwds_ultraback
        else:
            answer = fwds_curves
    for i in answer:
        yield(dex.upper(),i)
        
def bond_spline_presets(bond,build):
    answer = {'Outright': ['2y','5y','10y','30y'],
             'Curves': ['2s5s','2s10s','5s10s','5s30s','10s30s'], 
              'Flys':['2s5s10s', '5s10s30s']}
    for i in answer[build]:
        yield (bond.upper(), i)
        
def asw_spline_presets(bond,build,euro = 'ESTR'):
    answer = {'Outright': ['2y','5y','10y','30y'],
             'Curves': ['2s5s','2s10s','5s10s','5s30s','10s30s'], 
              'Flys':['2s5s10s', '5s10s30s']}
    for i in answer[build]:
        yield (bond.upper(), i,euro)

def custom_presets(broker):
    if broker == "MR Presets":
        return {'SWAP': [('USD','2y1y/3y2y/5y2y'),('GBP','2y1y/3y2y/5y2y'),('EUR','2y1y/3y2y/5y2y'),('EUR_6','2y1y/3y2y/5y2y'),
                         ('USD','5y2y/7y3y'),('GBP','5y2y/7y3y'),('EUR','5y2y/7y3y'),('EUR_6','5y2y/7y3y'),
                         ('USD','5s7s10s'),('GBP','5s7s10s'),('EUR','5s7s10s'),('EUR_6','5s7s10s'),
                        ]}
        

In [5]:
trades = {'SWAP':[],'FUTURE':[],'BOND':[], 'BOND FUTURE': [],'ASW':[],'MD SWAP': [] ,'XMKT SWAP':[],'FX':[] ,'OTHER/CIX': [], 'MY CIX':[]}
master = pd.DataFrame({})


In [6]:
from IPython.display import HTML, display
import ipywidgets as widgets
import pandas as pd

# Load custom CSS
display(HTML('''
<style>
.centered-content {
    display: flex;
    flex-direction: column;
    justify-content: center;
    align-items: center;
    text-align: center;
    width: 100%;
}

.centered-output {
    display: flex;
    justify-content: center;
    align-items: center;
    width: 100%;
    margin-top: 20px;
}

.centered-table {
    margin: 0 auto;
    text-align: center;
}
</style>
'''))

display(HTML('<div id="target-section"></div>'))

# Widgets for the first box
x = ['-'] + [i for i in trades.keys()]
assetW = widgets.Dropdown(options=x, description='Instrument')

output1 = widgets.Output()
output2 = widgets.Output()
outputn = widgets.Output()

def build_data_list(asset):
    eugov = ['DBR', 'SPGB', 'FRTR', 'BTP', 'BTP-DBR', 'FRTR-DBR', 'SPGB-DBR', 'BTP-SPGB', 'FRTR-SPGB']
    cix_options = get_cix_options()
    style = {'description_width': 'initial'}
    b_or_i = {
        '-': widgets.Text(value='Choose Instrument', disabled=True),
        'SWAP': widgets.Dropdown(options=[i.upper() for i in spot_library.keys()], description='Currency'),
        'FUTURE': widgets.Dropdown(options=['USD', 'EUR', 'GBP'], description='Currency'),
        'BOND': widgets.Dropdown(options=['UST', 'DBR', 'SPGB', 'FRTR', 'BTP', 'UKT', 'BTP-DBR', 'FRTR-DBR', 'SPGB-DBR', 'BTP-SPGB', 'FRTR-SPGB'], description='Bond'),
        'BOND FUTURE': widgets.Dropdown(options=['Yield', 'Price'], description='Field'),
        'MD SWAP': widgets.Dropdown(options=['FOMC', 'ECB', 'MPC'], description='Central Bank'),
        'ASW': widgets.Dropdown(options=['UST', 'DBR', 'SPGB', 'FRTR', 'BTP', 'UKT', 'BTP-DBR', 'FRTR-DBR', 'SPGB-DBR', 'BTP-SPGB', 'FRTR-SPGB'], description='Bond'),
        'FX': widgets.Text(description='Pair'),
        'XMKT SWAP': widgets.Dropdown(options=[i.upper() for i in spot_library.keys()], description='Currency'),
        'OTHER/CIX': widgets.Text(description='Ticker'),
        'MY CIX': widgets.Dropdown(options=cix_options, description='Ticker')
    }

    what_trade = {
        '-': widgets.Text(value='Choose Instrument', disabled=True),
        'SWAP': widgets.Text(description='Structure'),
        'FUTURE': widgets.Text(description='Structure'),
        'BOND': widgets.Text(description='Structure'),
        'BOND FUTURE': widgets.Text(description='Structure'),
        'ASW': widgets.Text(description='Structure'),
        'MD SWAP': widgets.Text(description='Structure'),
        'XMKT SWAP': widgets.Text(description='Structure'),
        'FX': widgets.Text(value=' ', disabled=True),
        'OTHER/CIX': widgets.Dropdown(options=['', 'Last Price', 'YTM'], description='Attribute'),
        'MY CIX': widgets.Dropdown(options=['', 'Last Price', 'YTM'], description='Attribute')
    }

    what_trade_p = {
        '-': widgets.Text(value='Choose Instrument', disabled=True),
        'SWAP': widgets.Dropdown(options=['-'] + swap_ops, description='Preset'),
        'FUTURE': widgets.Dropdown(options=['-'] + clr_dd, description='Preset'),
        'BOND': widgets.Dropdown(options=['-'] + bond_ops, description='Preset'),
        'BOND FUTURE': widgets.Text(value='-', disabled=True),
        'ASW': widgets.Dropdown(options=['-'] + bond_ops, description='Preset'),
        'MD SWAP': widgets.Text(value='-', disabled=True),
        'XMKT SWAP': widgets.Text(value='-', disabled=True),
        'FX': widgets.Text(value='-', disabled=True),
        'OTHER/CIX': widgets.Dropdown(options=['-', "MR Presets"], description='Preset'),
        'MY CIX': widgets.Dropdown(options=['-', "MR Presets"], description='Preset')
    }

    final = {
        '-': widgets.Text(value='Choose Instrument', disabled=True),
        'SWAP': widgets.Text(value='-', disabled=True),
        'FUTURE': widgets.Text(value='-', disabled=True),
        'BOND': widgets.Text(value='-', disabled=True),
        'BOND FUTURE': widgets.Text(value='-', disabled=True),
        'ASW': widgets.Dropdown(options=['ESTR', 'EURIBOR'], description='Against (if EUGOV)', style=style),
        'MD SWAP': widgets.Text(value='-', disabled=True),
        'XMKT SWAP': widgets.Dropdown(options=[i.upper() for i in spot_library.keys()], description='Against'),
        'FX': widgets.Text(value='-', disabled=True),
        'OTHER/CIX': widgets.Text(description='Label'),
        'MY CIX': widgets.Text(description='Label')
    }

    include_in_list = widgets.Button(description='Add Structure')
    removefrom_list = widgets.Button(description='Remove Last')
    include__preset = widgets.Button(description='Add Preset')

    global trades

    def on_button_clicked1(b):
        with output1:
            if asset == '-' or what_trade[asset].value == '':
                pass
            else:
                if asset in ['XMKT SWAP', 'OTHER/CIX', 'ASW', 'MY CIX']:
                    trades[asset].append((b_or_i[asset].value, what_trade[asset].value, final[asset].value))
                elif asset == 'BOND FUTURE':
                    trades[asset].append((what_trade[asset].value, b_or_i[asset].value))
                else:
                    trades[asset].append((b_or_i[asset].value, what_trade[asset].value))

                assetW.value = '-'

    def on_button_clickedn(b):
        with outputn:
            if asset == '-' or what_trade_p[asset].value == '-':
                pass
            else:
                if asset == 'FUTURE':
                    for tup in fut_presets(b_or_i[asset].value, what_trade_p[asset].value):
                        trades[asset].append(tup)
                elif asset == 'SWAP':
                    for tup in swap_presets(b_or_i[asset].value, what_trade_p[asset].value):
                        trades[asset].append(tup)
                elif asset == 'BOND':
                    for tup in bond_spline_presets(b_or_i[asset].value, what_trade_p[asset].value):
                        trades[asset].append(tup)
                elif asset == 'ASW':
                    for tup in asw_spline_presets(b_or_i[asset].value, what_trade_p[asset].value, final[asset].value):
                        trades[asset].append(tup)
                elif asset == 'OTHER/CIX' or asset == 'MY CIX':
                    custom_trades = custom_presets(what_trade_p[asset].value)
                    for a in custom_trades.keys():
                        trades[a] += custom_trades[a]

                assetW.value = '-'

    def on_button_clicked2(b):
        with output2:
            if assetW == '-':
                pass
            else:
                trades[asset].pop()
                assetW.value = '-'

    # Wrap widgets in centered box
    centered_box = widgets.VBox([
        widgets.HBox([b_or_i[asset], what_trade[asset], final[asset]]),
        what_trade_p[asset],
        widgets.HBox([include_in_list, removefrom_list, include__preset])
    ], layout=widgets.Layout(display='flex', justify_content='center', align_items='center', width='100%'))

    display(centered_box)

    include_in_list.on_click(on_button_clicked1)
    removefrom_list.on_click(on_button_clicked2)
    include__preset.on_click(on_button_clickedn)

    max_len = max([len(trades[i]) for i in trades.keys()])
    y = {}
    for i in trades.keys():
        #print(trades)
        if i == 'XMKT SWAP':
            y[i] = [f'{i[0]}-{i[2]} {i[1]}' for i in trades[i]] + [float('nan')] * (max_len - len(trades[i]))
        elif i == 'ASW':
            y[i] = [f'{i[0]}-{i[1]} v {i[2]}' if i[0] in eugov else f'{i[0]} {i[1]}' for i in trades[i]] + [float('nan')] * (max_len - len(trades[i]))
        elif i == 'BOND FUTURE':
            y[i] = [j[0] for j in trades[i]] + [float('nan')] * (max_len - len(trades[i]))
        elif i == 'OTHER/CIX' or i == 'MY CIX':
            y[i] = [f'{i[2]}' for i in trades[i]] + [float('nan')] * (max_len - len(trades[i]))
        else:
            y[i] = [f'{i[0]} {i[1]}' for i in trades[i]] + [float('nan')] * (max_len - len(trades[i]))

    styled_table = pd.DataFrame(y).style.format('{}', na_rep='').set_table_styles([{
        'selector': 'table',
        'props': [('margin-left', 'auto'), ('margin-right', 'auto'), ('text-align', 'center')]
    }]).to_html()
    
    centered_html = f'<div class="centered-output"><div class="centered-table">{styled_table}</div></div>'
    display(HTML(centered_html))

interactive(build_data_list, asset=assetW)


interactive(children=(Dropdown(description='Instrument', options=('-', 'SWAP', 'FUTURE', 'BOND', 'BOND FUTURE'…

In [11]:
df = blp.bdh("GTGBP5Y Govt", flds='px_last', start_date="01/01/2019",end_date="16/07/2024").fillna(method ='bfill') 
#base = blp.bdh(tickers, 'px_last', start, end).fillna(method ='bfill') * 100

In [12]:
df

,GTGBP5Y Govt
,px_last
2019-01-01,0.8997
2019-01-02,0.8328
2019-01-03,0.8220
2019-01-04,0.8916
2019-01-07,0.8851
...,...
2024-07-10,3.9768
2024-07-11,3.9185
2024-07-12,3.9454


In [ ]:
# Load custom CSS
display(HTML('''
<style>
.centered-content {
    display: flex;
    flex-direction: column;
    justify-content: center;
    align-items: center;
    text-align: center;
    width: 100%;
}

.centered-output {
    display: flex;
    justify-content: center;
    align-items: center;
    width: 100%;
    margin-top: 20px;
}

.centered-table {
    margin: 0 auto;
    text-align: center;
}
</style>
'''))

output3 = widgets.Output()
get_data_frombb = widgets.Button(description='Pull BBG Data')
gen_from = widgets.DatePicker(value=dt(dt.today().year - 1, dt.today().month, dt.today().day), description='Start')
gen_to = widgets.DatePicker(value=dt.today(), description='End')

# Center the DatePicker widgets and Button
centered_box = widgets.VBox([
    widgets.HBox([gen_from, gen_to], layout=widgets.Layout(display='flex', justify_content='center', align_items='center', width='100%')),
    widgets.HBox([get_data_frombb], layout=widgets.Layout(display='flex', justify_content='center', align_items='center', width='100%')),
    output3
], layout=widgets.Layout(display='flex', justify_content='center', align_items='center', width='100%'))

display(centered_box)

def on_button_clicked3(b):
    global master
    with output3:
        output3.clear_output()
        master = function_for_bb(trades, gen_from.value, gen_to.value)
        master.to_csv('Data1.csv')
        #print(master)
        dis = pd.DataFrame({}, index=['CURRENT', 'AVG MOVE', 'DAILY VOL'])
        dis.loc['CURRENT', master.columns] = master.loc[master.index[-1]]
        dis.loc['AVG MOVE', master.columns] = (master.diff().mean())
        dis.loc['DAILY VOL', master.columns] = (master.diff().std())

        # Center the table output using a div wrapper
        styled_table = dis.style.set_table_styles([{
            'selector': 'table',
            'props': [('margin-left', 'auto'), ('margin-right', 'auto'), ('text-align', 'center')]
        }]).to_html()
        
        centered_html = f'<div class="centered-output"><div class="centered-table">{styled_table}</div></div>'
        output3.append_display_data(HTML(centered_html))

get_data_frombb.on_click(on_button_clicked3)

Button(description='Pull BBG Data', style=ButtonStyle())

Output()

## Z-Score

In [ ]:
from scipy import stats
from scipy.optimize import minimize 
from scipy.special import ndtr
from datetime import datetime as dt, timedelta
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

output_z = widgets.Output()
show_z = widgets.Button(description='Show Z-Scores')
WINDOWS = {'1W': 5, '2W': 10, '1M': 20, '3M': 60, '6M': 120, '1Y': 250}

gen_from_z = widgets.DatePicker(value=dt(dt.today().year - 1, dt.today().month, dt.today().day), description='Start')
gen_to_z = widgets.DatePicker(value=dt.today()+-timedelta(days=1), description='End')

study_target_z = widgets.Dropdown(options=list(master.columns), description='Target')
update_dropdown_z = widgets.Button(description='Update Dropdown')

# Center the DatePicker widgets and Button
centered_box = widgets.VBox([
    widgets.HBox([gen_from_z, gen_to_z], layout=widgets.Layout(display='flex', justify_content='center', align_items='center', width='100%')),
    widgets.HBox([study_target_z, show_z], layout=widgets.Layout(display='flex', justify_content='center', align_items='center', width='100%')),
    output_z
], layout=widgets.Layout(display='flex', justify_content='center', align_items='center', width='100%'))

display(centered_box)
display(update_dropdown_z)

def on_button_clicked_z(b):
    with output_z:
        output_z.clear_output()
        master_z = function_for_bb(trades, gen_from_z.value, gen_to_z.value)
        master_z = master_z[[study_target_z.value]]

        #print(master_z)
        # Convert the index to date if it's datetime
        master_z.index = pd.to_datetime(master_z.index).date

        # Ensure gen_from_z.value and gen_to_z.value are dates
        start_date = gen_from_z.value
        end_date = gen_to_z.value
        #subset = master_z[(master_z.index >= start_date)&(master_z.index <= end_date)]
        subset = master_z.diff()

        z_table = pd.DataFrame(index=master_z.columns)
        
        z_table['LEVEL'] = master_z.iloc[-1].values
        z_move = subset.iloc[-1]
        z_table['AVERAGE MOVE'] = subset.mean().values
        z_table['DAILY VOL'] = subset.std().values
        z_table['Z-SCORE'] = (z_move - z_table['AVERAGE MOVE']) / z_table['DAILY VOL']
        z_table['PERCENTILE'] = [stats.norm.cdf(i) for i in z_table['Z-SCORE']]

        # Center the table output using a div wrapper
        styled_table = z_table.style.format({
            'LEVEL': '{:.2f} bps',
            'EVENT DAY MOVE': '{:.2f} bps',
            'AVERAGE MOVE': '{:.2f} bps',
            'DAILY VOL': '{:.2f} bps',
            'Z-SCORE': '{:.2f}',
            'PERCENTILE': '{:.1%}',
        }).set_table_styles([{
            'selector': 'table',
            'props': [('margin-left', 'auto'), ('margin-right', 'auto'), ('text-align', 'center')]
        }])

        display(styled_table)

def on_button_clicked_ud_z(b):
    study_target_z.options = ['-'] + list(master.columns)

# Assign event handlers to buttons
show_z.on_click(on_button_clicked_z)
update_dropdown_z.on_click(on_button_clicked_ud_z)